# RESNET Model

ResNet's design excels in deep learning due to its modular structure and residual connections. These elements allow for easy deepening of the network without complexity. The modularity of ResNet makes it straightforward and efficient, enabling it to integrate smoothly with other neural networks. This combination of modularity and residual connections makes ResNet both powerful and versatile for enhancing deep neural network models.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf

import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras import layers, models
from keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, GlobalAveragePooling2D, Dense, Layer


# Results & Method

Train Accuracy: 97.37%

Test Accuracy: 94.95%

  The accuracy for ResNet indicate a high level of performance, especially considering the small image size of 30x30 pixels.Due to the small image size, In our experiment, the utilized ResNet architecture has only half blocks of the original ResNet.
  
  Our team opted not to utilize pre-trained weights in our experiment since the ResNet structure we employed is only half as deep as the original ResNet, resulting in a significantly reduced number of weight parameters. The lesser complexity of our network means that the pre-trained weights, which are tailored for a denser parameter distribution in deeper models, might not transfer effectively to our more streamlined architecture. 

In [28]:
def resnet_identity_block(x, filters, kernel_size):
    """A ResNet identity block with two Conv2D layers."""
    fx = Conv2D(filters, kernel_size, padding='same')(x)
    fx = BatchNormalization()(fx)
    fx = Activation('relu')(fx)

    fx = Conv2D(filters, kernel_size, padding='same')(fx)
    fx = BatchNormalization()(fx)

    out = Add()([x, fx])  # Add() layer creates a residual connection
    out = Activation('relu')(out)
    return out

def resnet_conv_block(x, filters, kernel_size, strides):
    """A ResNet convolutional block with a Conv2D shortcut."""
    fx = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    fx = BatchNormalization()(fx)
    fx = Activation('relu')(fx)

    fx = Conv2D(filters, kernel_size, padding='same')(fx)
    fx = BatchNormalization()(fx)

    shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same')(x)
    shortcut = BatchNormalization()(shortcut)

    out = Add()([fx, shortcut])  # Shortcut connection
    out = Activation('relu')(out)
    return out

# Input layer
input_shape = (30, 30, 3)
inputs = Input(shape=input_shape)

# Initial Conv2D layer
x = Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2))(x)

# Residual blocks
x = resnet_conv_block(x, 64, (3, 3), strides=(1, 1))
x = resnet_identity_block(x, 64, (3, 3))
x = MaxPooling2D((2, 2))(x)

x = resnet_conv_block(x, 128, (3, 3), strides=(1, 1))
x = resnet_identity_block(x, 128, (3, 3))

# Finishing layers
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dense(106, activation='softmax')(x)  # Adjust the number of classes

# Create the model
RESNET_model = Model(inputs=inputs, outputs=x)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 30, 30, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 30, 30, 32)           896       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 15, 15, 64)           18496     ['max_pooling2d[0][0]']   

                                                                                                  
 activation_6 (Activation)   (None, 7, 7, 128)            0         ['batch_normalization_8[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_10 (Conv2D)          (None, 7, 7, 128)            147584    ['activation_6[0][0]']        
                                                                                                  
 batch_normalization_9 (Bat  (None, 7, 7, 128)            512       ['conv2d_10[0][0]']           
 chNormalization)                                                                                 
                                                                                                  
 add_3 (Add)                 (None, 7, 7, 128)            0         ['activation_5[0][0]',        
          